# Text data

как обработка естественного языка (англ. Natural Language Processing, NLP). NLP изучает проблемы компьютерного анализа естественных языков - т.е. языков, которые для общения используют люди (а не придуманных искусственно (например, азбука Морзе - язык, придуманный искусственно). Поговорим подробнее о том, зачем нужен NLP и где именно возникает задача обработки естественного языка.

Тексты - один из самых доступных и объёмных источников данных: легко собирать и просто хранить

Например, если у вас интернет-магазин, то для анализа доступны

* текстовые описания товаров
* пользовательские комментарии
* диалоги с продавцом-консультантом в чатике

Текстовую информацию просто хранить, поэтому проекты накапливают огромные наборы данных такого рода и очень хотят извлекать из этих объёмов полезную информацию.

Как специалист по ML в начале карьеры вы, скорее всего, встретите ряд “классических” задач - например, определение тональности (настроения) текста или классификации сообщений spam/not spam - для таких задач используются подходы, основанные на подсчёте статистик по встречающимся в тексте словам.
Однако, есть и другие, более сложные задачи.

Для решения применяются различные архитектуры нейросетей (RNN, LSTM) - это мощные инструменты, которые позволяют решать сложные задачи, например:

* извлечения именованных сущностей ([NER](https://habr.com/ru/post/414175/), Named-Entity Recognizing)
* автоматизированного перевода (например, сервис *google translate* производит перевод с помощью глубоких сетей)
* Speech Recognition - распознавание речи, трансляция из аудио в текстовый вид
* Natural Language Generation - генерация текстов, например можно генерировать подписи к картинкам

У обработки естественного языка есть ряд особенностей:

* необходимо размечать большой объём данных для обучения с учителем. Допустим, хотим отделять спам-сообщения от не спама. Вам нужно найти людей, которые прочитают все смс, которые удалось собрать и отметят те из них, которые являются спамом - текстов обычно очень много и разметка данных может оказаться дорогим удовольствием
* модель, обученную на одном языке невозможно использовать для другого языка
* важен как синтаксис, так и семантика (смысл). Например, во фразе: «Вот списки студентов, которые сдали зачет по физике» определение «которые сдали зачет по физике» относится к студентам, а в предложении: «Вот списки студентов, которые лежали в шкафу у декана»  структура фразы (тот самый синтаксис) такая же, как и предыдущей - но определение уже относится не к студентам, а к листкам бумаги. От компьютера мы хотим добиться, чтобы смыл обеих фраз был “понят” одинаково хорошо.

Кроме того, для текстов на естественном языке довольно сложно проводить предобработку данных, этот этап сильно зависит от задачи, которую вы  решаете. Так, например, для задачи анализа тональности текста знаки препинания, скорее всего, не важны. Однако, для задачи извлечения именованных сущностей (именованная сущность - это имя собственное - например название организации или географического объекта) удалять знаки препинания не рекомендуется - это может привести к потере важной информации. Например если из фразы `Мы пошли обедать в “Берёзку”` если удалить все знаки препинания (кавычки) и заглавную букву в названии заведения то станет сложнее понять, что речь идёт о кафе.

Обработка текста складывается из двух этапов

* предварительная обработка текста
* векторизация текста


#### Предварительная обработка текста

Перед тем, как обучать модель, данные следует специальным образом подготовить. Подготовка данных включает в себя несколько обязательных этапов

* удалить все нерелевантные символы (например, любые символы, не относящиеся к цифро-буквенным).
* токенизировать текст, разделив его на индивидуальные слова (токены)
* удалить нерелевантные слова — например, упоминания в Twitter или URL-ы.
* перевести все символы в нижний регистр для того, чтобы слова «hello», «Hello» и «HELLO» были схлопнуты в один токен
* исправление ошибок ("молоко" и "молако" - одно слово, но разные токены, не надо так)  
* лемматизация - перевод слова в нормальную (словарную) форму (например, «машина» вместо «машиной»). Существительные должны быть приведены к единственному числу именительного падежа, глаголы - инфинитив и т.д.
* стемминг - процедура, когда от слова переходим к его корню ("помыть" и "мытый" - корень "мыт"). То есть все "помытые" заменяем на "мыт".

Все эти приёмы нужно применять с осторожностью и внимательно следить за тем, как тот или иной приём, применённый к исходному тексту, влияет на качество решения задачи (например, выявлению спама)

Для демонстрации всех этих приёмов загрузим корпус (набор текстов) с твитами о продуктах. Для каждого твита размечена эмоциональная окраска - позитивная, нейтральная или негативная. Примечание: для  обработки текста воспользуемся библиотекой nltk, которая [доступна в anaconda](https://anaconda.org/anaconda/nltk)

Загружаем датасет с результатами модерации контента

In [1]:
import os

run_env = os.getenv('RUN_ENV', 'COLLAB')
if run_env == 'COLLAB':
  from google.colab import drive
  ROOT_DIR = '/content/drive'
  drive.mount(ROOT_DIR)
  print('Google drive connected')
  DRIVE_DATA_DIR = 'ml_course_data'
  root_data_dir = os.path.join(ROOT_DIR, 'MyDrive', DRIVE_DATA_DIR)
else:
  root_data_dir = os.getenv('DATA_DIR', '/srv/data')

if not os.path.exists(root_data_dir):
  raise RuntimeError('Отсутствует директория с данными')
else:
  print('Содержимое директории %s: %s' % (root_data_dir, os.listdir(root_data_dir)[:5]))

Содержимое директории /Users/adzhumurat/PycharmProjects/ai_product_engineer/data: ['client_segmentation.csv', 'messages.db', 'labeled_data_corpus.csv', 'chroma', 'content_description.csv']


## Препроцессинг текста

Устанавливаем NLTK - библиотеку для обработки тестов. Для начала готовим директорию для справочников

In [2]:
nltk_data_dir = os.path.join(root_data_dir, 'nltk_data')
if not os.path.exists(nltk_data_dir):
  os.makedirs(nltk_data_dir)
  print(f'Директория {nltk_data_dir} создана')
logs_dir = os.path.join(root_data_dir, 'logs')
if not os.path.exists(logs_dir):
  os.makedirs(logs_dir)
print(f'Подготовили директорию для nltk {nltk_data_dir}')

Подготовили директорию для nltk /Users/adzhumurat/PycharmProjects/ai_product_engineer/data/nltk_data


Если работаете в коллабе надо будет установить пакет

In [3]:
from IPython.display import clear_output

if run_env == 'COLLAB':
    !pip install nltk==3.6.2
    # !pip install git+https://github.com/openai/CLIP.git
    clear_output()
print('Установили NLTK')

Установили NLTK


In [4]:
import nltk

nltk.download('punkt', download_dir=nltk_data_dir)
nltk.data.path.append(nltk_data_dir) # тут почему-то корневую надо указывать ¯\_(ツ)_/¯

[nltk_data] Downloading package punkt to /Users/adzhumurat/PycharmProj
[nltk_data]     ects/ai_product_engineer/data/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [5]:
import nltk

sample_text = """SoftBank Vision Fund 2 is leading the round, a Series C, with iPod “father” and Nest co-founder Tony Fadell (by way of Future Shape), Blisce, French entrepreneur Xavier Niel, Mirabaud, Cassius and Evolution — all previous backers — also participating. (Previous investors in the company also include DeepMind co-founders Mustafa Suleyman and Demis Hassabis, notable given the company’s early focus on data science and recommendation algorithms.) Prior to this round Dice had raised around $45 million, according to PitchBook estimates."""

print('== Исходный текст== \n%s\n\n' % sample_text)

tokenized_str = nltk.word_tokenize(sample_text)

print('== Токенизированный текст==\n%s' % tokenized_str)

== Исходный текст== 
SoftBank Vision Fund 2 is leading the round, a Series C, with iPod “father” and Nest co-founder Tony Fadell (by way of Future Shape), Blisce, French entrepreneur Xavier Niel, Mirabaud, Cassius and Evolution — all previous backers — also participating. (Previous investors in the company also include DeepMind co-founders Mustafa Suleyman and Demis Hassabis, notable given the company’s early focus on data science and recommendation algorithms.) Prior to this round Dice had raised around $45 million, according to PitchBook estimates.


== Токенизированный текст==
['SoftBank', 'Vision', 'Fund', '2', 'is', 'leading', 'the', 'round', ',', 'a', 'Series', 'C', ',', 'with', 'iPod', '“', 'father', '”', 'and', 'Nest', 'co-founder', 'Tony', 'Fadell', '(', 'by', 'way', 'of', 'Future', 'Shape', ')', ',', 'Blisce', ',', 'French', 'entrepreneur', 'Xavier', 'Niel', ',', 'Mirabaud', ',', 'Cassius', 'and', 'Evolution', '—', 'all', 'previous', 'backers', '—', 'also', 'participating', '

Отфильтруем знаки пунктуации, токены приведём к нижнему регистру

In [6]:
import string

tokens = [i.lower() for i in tokenized_str if ( i not in string.punctuation )]
print(tokens)

['softbank', 'vision', 'fund', '2', 'is', 'leading', 'the', 'round', 'a', 'series', 'c', 'with', 'ipod', '“', 'father', '”', 'and', 'nest', 'co-founder', 'tony', 'fadell', 'by', 'way', 'of', 'future', 'shape', 'blisce', 'french', 'entrepreneur', 'xavier', 'niel', 'mirabaud', 'cassius', 'and', 'evolution', '—', 'all', 'previous', 'backers', '—', 'also', 'participating', 'previous', 'investors', 'in', 'the', 'company', 'also', 'include', 'deepmind', 'co-founders', 'mustafa', 'suleyman', 'and', 'demis', 'hassabis', 'notable', 'given', 'the', 'company', '’', 's', 'early', 'focus', 'on', 'data', 'science', 'and', 'recommendation', 'algorithms', 'prior', 'to', 'this', 'round', 'dice', 'had', 'raised', 'around', '45', 'million', 'according', 'to', 'pitchbook', 'estimates']


Удаляем стоп-слова, список которых для русского языка можно получить как `stop_words = nltk.corpus.stopwords.words('russian')`. Стоп-слова это "мусорные" слова которые встречаются чрезычайно часто (в каждом предложении) поэтому не несут в себе никакой информации. Такие слова, вобщем-то, нужны только для красивой речи и поэтому можем их смело удалять из текста. Например, этот список стоп-слов я нагуглил в интернете.

In [13]:
stop_words = [
    'i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd",
    'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers',
    'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which',
    'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been',
    'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if',
    'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between',
    'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out',
    'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why',
    'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not',
    'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', "don't", 'shold',
    "should've", 'now', 'd', 'll', 'm', 'o', 're', 've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't",
    'didn', "didn't", 'doesn', "doesn't", 'hadn', "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't",
    'ma', 'mightn', "mightn't", 'mustn', "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't",
    'wasn', "wasn't", 'weren', "weren't", 'won', "won't", 'wouldn', "wouldn't"
]

filtered_tokens = [i for i in tokens if ( i not in stop_words )]

print(filtered_tokens)

['softbank', 'vision', 'fund', '2', 'leading', 'round', 'series', 'c', 'ipod', '“', 'father', '”', 'nest', 'co-founder', 'tony', 'fadell', 'way', 'future', 'shape', 'blisce', 'french', 'entrepreneur', 'xavier', 'niel', 'mirabaud', 'cassius', 'evolution', '—', 'previous', 'backers', '—', 'also', 'participating', 'previous', 'investors', 'company', 'also', 'include', 'deepmind', 'co-founders', 'mustafa', 'suleyman', 'demis', 'hassabis', 'notable', 'given', 'company', '’', 'early', 'focus', 'data', 'science', 'recommendation', 'algorithms', 'prior', 'round', 'dice', 'raised', 'around', '45', 'million', 'according', 'pitchbook', 'estimates']


In [14]:
import nltk
import string
import pandas as pd

# # дополнительный словарь со знаками пунктуации
# nltk.download('punkt', download_dir='.')

df = pd.read_csv(os.path.join(root_data_dir, 'brand_tweets.csv'), sep=',', encoding='utf8')
# удаляем строки, в которых отсутствует текст твита
df.drop(df[df['tweet_text'].isnull()].index, inplace=True)
print(df.shape)

df.head()

(3904, 3)


,tweet_text,emotion_in_tweet_is_directed_at,is_there_an_emotion_directed_at_a_brand_or_product
0,.@wesley83 I have a 3G iPhone. After 3 hrs twe...,iPhone,Negative emotion
1,@jessedee Know about @fludapp ? Awesome iPad/i...,iPad or iPhone App,Positive emotion
2,@swonderlin Can not wait for #iPad 2 also. The...,iPad,Positive emotion
3,@sxsw I hope this year's festival isn't as cra...,iPad or iPhone App,Negative emotion
4,@sxtxstate great stuff on Fri #SXSW: Marissa M...,Google,Positive emotion


Реализуем пайплайн в виде функции, при помощи которой обработаем все текстовые описания. Для каждого описания
* проводим токенизацию
* удаляем пунктуацию
* приводим к нижнему регистру
* удаляем стоп-слова


Примените процедуру токенизации к файлу brand_tweets.csv

Сколько уникальных токенов получилось?

In [15]:
def tokenize_text(raw_text: str):
    """Функция для токенизации текста

    :param raw_text: исходная текстовая строка
    """
    filtered_tokens = []
    # -- ВАШ КОД ТУТ --

    filtered_tokens = [
        i.lower() for i in raw_text.split() if ( i not in string.punctuation )
    ]

    # TODO: фильтрация стоп-слов
    # TODO: удаляем короткие токены меньше трех символов

    # -----------------
    return filtered_tokens

# применяем функцию в датафрейму с помощью метода .apply()
tokenized_tweets= df['tweet_text'].apply(tokenize_text)

# добавляем новую колонку в исходный датафрейм
df = df.assign(
    tokenized=tokenized_tweets
)

df['tokenized'].head()

0    [.@wesley83, i, have, a, 3g, iphone., after, 3...
1    [@jessedee, know, about, @fludapp, awesome, ip...
2    [@swonderlin, can, not, wait, for, #ipad, 2, a...
3    [@sxsw, i, hope, this, year's, festival, isn't...
4    [@sxtxstate, great, stuff, on, fri, #sxsw:, ma...
Name: tokenized, dtype: object

Следующий шаг - привести слово в нормальную (словарную) форму. Для русского языка можно проводить нормализацию можно с помощью модуля **pyMorphy**

```python
import pymorphy2

morph = pymorphy2.MorphAnalyzer()
parsed_token = morph.parse(word)
normal_form = parsed_token[0].normal_form
```

В силу того, что наши твиты на английском языке, то этап нормализации не слишком актуален.


**NOTE**: изучите самостоятельно `jupyter_notebooks/vol_04_deep_dive_00_probability_hw_2_naive_bayes.ipynb` - этот демо-файл показывает, как использовать простые пайплайны препроцессинга текстов для создания наивного байесовского классификатора


### Векторизация текста: Bag of Words


Мы умеем подготавливать текст к обработке: приводить слова к начальным формам, разделять текст на токены, удалять "мусорные" токены (стоп-слова). Однако, мы знаем, что нейросети работают не с текстом, а с числами. Давайте разбираться, как переводить токены в числа, то есть с тем, как работает векторизация

Bag of Words - это способ перейти от набора токенов к численному вектору. Алгоритм векторизации текста по модели BoW:

1. определяем количество $N$ различных токенов во всех доступных текста - так называемый "словарь"
1. присваиваем каждому токену случайный номер от $0$ до $N$
1. для каждого документа $i$ формируем вектор размерности $N$ - ставим на позицию $j$ количество вхождений токена с номером $j$, которые содержатся в тексте $i$.

Каждый токен мы по сути представляем в виде вектора размерности $N$, который состоит из нулей и всего одной единицы, такое кодирование называется *One-Hot encoding*. А каждый документ это "сумма" всех one-hot векторов входящих в него токенов

Такой подход хорошо иллюстрируется картинкой:

![bow](img/bow.png)

Для каждого элемента получаем вектор из нулей и единиц. При этом размерность словаря обычно составляет несколько десятков тысяч, а количество токенов в одном документе несколько десятков - то есть нулей значительно больше, чем единиц - такие данные называются *разреженными*.

В таком виде данные уже пригодны для работы с нейросетью или любым другим алгоритмом ML, однако есть несколько довольно простых и полезных вещей, которые мы можем сделать и без нейросетей. Давайте сначала разберем их, а потом вернемся к нейросетям. Такое представление текста позволяет решать интересные задачи - например, находить самые похожие друг на друга тексты. Чтобы как-то формализовать понятие "схожести" текстов, вводится понятие *косинусного расстояния* между двумя векторами текстов $a$ и $b$ размерности $N$. С этой метрикой вы [можете познакомиться в Википедии](https://ru.wikipedia.org/wiki/Векторная_модель#Косинусное_сходство ), формула такая для двух векторов $a$ и $b$ с координатами $a_i$ и $b_i$ соответственно:
$$
\text{similarity} = \cos (\theta) = 1 - \frac{\sum_{i=1}^{N}a_ib_i}{\sqrt{\sum_{i=1}^{N}(a_i)^2}\sqrt{\sum_{i=1}^{N}(b_i)^2}}
$$

Интуитивное объяснение для простого случая: два документа полностью совпадают, тогда единички в них стоят на одних и тех же местах - расстояние между ними будет нулевым. Если два текста совершенно не пересекаются, то единички будут стоять на разных местах - расстояние в этом случае равно единице. Самостоятельно реализовывать функцию не нужно - есть готовая реализация в [scipy.spatial.distance.cosine](https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.spatial.distance.cosine.html)

Векторизуем наш корпус (набор текстов) с помощью класса `CountVectorizer()` (то есть превращаем наборы токенов в наборы векторов)

In [16]:
from sklearn.feature_extraction.text import CountVectorizer

# инициализируем объект, который токенизирует наш текст
# в качестве единственного аргимента передаём функцию, которую мы написали в Уроке 2
# на разбивает каждый документ на токены
vectorizer = CountVectorizer(tokenizer=tokenize_text)
# применяем наш объект-токенизатор к датафрейму с твитами
document_matrix = vectorizer.fit_transform(df['tweet_text'].values)
# результат - матрица, в которой находятся числа, строк в мастрице столько, сколько документов
# а столбцов столько, сколько токенов
document_matrix

/Users/adzhumurat/PycharmProjects/ai_product_engineer/.venv/lib/python3.12/site-packages/sklearn/feature_extraction/text.py:523: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


<Compressed Sparse Row sparse matrix of dtype 'int64'
	with 64190 stored elements and shape (3904, 10795)>

In [17]:
# vectorizer.get_feature_names_out()

Класс `sklearn.feature_extraction.text.CountVectorizer` реализует алгоритм преобразования массива текстовых документов в разреженную матрицу такую, что

* число строк совпадает с количеством документов в исходном датафрейме
* количество столбцов совпадает с количеством различных токенов
* объект `CountVectorizer()` содержит в себе разные вспомогательные элементы - например, словарь соответствия токена и его номера

Полученные вектора можно использовать в алгоритмах второго уровня - например, в задаче классификации отзывов.

Пользуясь матрицей, найдем твит, который максимально похож на первый твит из набора

Вычисляем попарные схожести между элементами разреженной матрицы:

In [18]:
from sklearn.metrics import pairwise_distances

tweet_distance = 1 - pairwise_distances(document_matrix, metric="cosine")

tweet_distance.shape

(3904, 3904)

Мы получили квадратную матрицy, которая содержит столько строк и столбцов, сколько документов в нашем  корпусе  (наборе текстов).

In [19]:
import numpy as np
TARGET_INDEX = 827


# отсортируем твиты по “похожести” - чем похожее на source_tweet_index,
# тем ближе к началу списка sorted_similarity
sorted_similarity = np.argsort(-tweet_distance[TARGET_INDEX,:])

sorted_similarity

array([ 827,  929, 1523, ..., 1190, 1103, 2850], shape=(3904,))

Мы получили квадратную матрицy, которая содержит столько строк и столбцов, сколько документов в нашем  корпусе  (наборе текстов).

In [20]:
import numpy as np

# отсортируем твиты по “похожести” - чем похожее на source_tweet_index,
# тем ближе к началу списка sorted_similarity
sorted_similarity = np.argsort(-tweet_distance[TARGET_INDEX,:])

sorted_similarity

array([ 827,  929, 1523, ..., 1190, 1103, 2850], shape=(3904,))

Мы получили вектор "схожестей", который содержит индексы похожих твитов, расположенных по убыванию схожести. Больше всего твит похож сам на себя, поэтому возьмём индекс второго по схожести элемента (и далее).

In [21]:

print(df.iloc[TARGET_INDEX].tweet_text)
print('-------------')
print(df.iloc[sorted_similarity[1]].tweet_text)
print('-------------')
print(df.iloc[sorted_similarity[2]].tweet_text)
print('-------------')
print(df.iloc[sorted_similarity[3]].tweet_text)

Is Google launching its own social network? Definitely not at ##sxsw. But rumor still has it...{link}  #googlecircles
-------------
Google is NOT launching any products at #SXSW {link}
-------------
@mention ‰ЫП@mention Google is NOT launching any products at #SXSW {link}
-------------
Google to launch social network Google Circles  at some point - but not at SXSW today.  :-(   {link}   #sxsw


Мы получили мощный инструмент для анализа текстов - например, мы случайно нашли дубликат твита

Кроме простого подхода, когда мы вычисляем счётчик вхождения токена, можно вычислять более сложную метрику TF-IDF (term frequency - inverse document frequency), которая вычисляется по следующей формуле для токена $t$ и документа $d$:
$$
\text{tf-idf}(t,d) = \text{tf}(t,d)\cdot\text{idf}(t)
$$

Где $\text{tf}(t,d)$ - элемент матрицы, полученной из `CountVectorizer()`, который мы умножаем на величину $\text{idf}(t)$.

Этот класс тоже реализован в sklearn, его предлагаю использовать в домашней работе
```python
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(
    analyzer='word',
    lowercase=True,
    token_pattern=r'\b[\w\d]{3,}\b',
    min_df=0.001
)

```

Эта величина показывает количество документов в корпусе  (наборе текстов), в которых был встречен токен $t$:
$$
\text{idf}(t) = \log\frac{1+N}{1+\text{df(t)}} + 1
$$

где $\text{df}(t)$ - количество документов корпуса, в которых был встречен токен $t$. Таким образом мы понижаем веса у слов, которые встречаются почти во всех документах - такие токены являются неинформативными и мусорными, алгоритм понижает их "важность" для анализа.

Алгоритм TF-IDF лучше подходит для анализа текстов и даёт более высокое качество, но более затратен по вычислениям. Как выбрать между этими алгоритмами?

* если токенов менее 10000 используйте TF-IDF
* если токенов более 10000 то *попробуйте* использовать TF-IDF, если не получится - возвращайтесь к CountVectorizer

**Недостатки BoW подхода** Используя алгоритмы вроде Вag of Words, мы теряем порядок слов в тексте, а значит, тексты "i have no cows" и "no, i have cows" будут идентичными после векторизации, хотя и противоположными семантически. Чтобы избежать этой проблемы, можно сделать шаг назад и изменить подход к токенизации: например, использовать N-граммы (комбинации из N последовательных токенов). Обычно по корпусу  (набору текстов) формируются биграммы (последовательности из двух слов) или триграммы (последовательности из трёх слов)

Кроме того, текст можно разбивать не на слова, а на последовательности букв (биграммы, триграммы) - при таком подходе опечатки будут автоматически учитываться.


## Другие подходы к векторизации текста: Word2Vec

Это более новый алгоритм, чем BoW.

Алгоритм предполагает, что мы  уже разбили документы на токены и готовы скормить наши токены нейросети, которая сделает из каждого токена плотный вектор-эмбеддинг.

При подходе BoW мы представляем каждый документ в виде разреженного вектора-строки, где размерность вектора соответствует количеству токенов в словаре.

Нетрудно заметить, что при таком подходе игнорируется контекст, в котором находится слово. Например, в двух предложениях "король издал указ" и  "правитель издал указ" слова *король* и *правитель* являются синонимами, потому что используются в одинаковом контекста. Подход *BoW* не сможет уловить отношение синонимии.

Эту проблему решает подход `Word Embedding`, при котором каждое слово представляет собой вектор большой размерности (обычно несколько сотен). В отличие от подхода BoW, при котором каждое слово представляет собой разреженный вектор, *word embedding* - это "плотный" вектор. Классическим алгоритмом, вычисляющим эмбеддинги (то есть "плотные" вектора) слов, является Word2Vec, предложена чешским аспирантом Томашем Миколовым в 2013 году. Эта модель позволяет формировать векторы, которые отражают взаимоотношения между словами: "король" относится к "королеве" так же как "женщина" к "мужчине"

![word_vectors](img/word_vectors.png)

Подход Word2Vec основан на интуитивно понятной гипотезе, которая называется гипотезой локальности — "слова, которые встречаются в одинаковых окружениях, имеют близкие значения". Эта гипотеза приводит к двум способам тренировки моделей: *Continious Bag of Words* (когда по контексту предсказываем слово) и *Skip Gram* - когда по слову пытаемся предсказать его контекст. Эмбеддинги, полученные с помощью обоих подходов оказываются идентичными - можно применять любой из них.

Пример контекста:

*Машинное обучение это* **класс** *методов искусственного интеллекта*

Мы видим, что из текста вырезается окно текста, слово в центре окна мы хотим предсказать, используя слова по краям "окна" (тот самый *контекст*).

На схеме представлены оба подхода:

![word2vec](img/word2vec.png)

На картинке представлен алгоритм тренировки *W2V* в виде простой нейросети:
![w2v_net](img/w2v_net.png)

На схеме слева-направо:

* Входной вектор $(x_1,\ldots,x_v)$ - слово из словаря, закодированное One-Hot
* $W_{V\times N}$ - матрица *word input* -  это эмбеддинги, которые мы обучаем
* Эмбеддинг слова контекста $(h_1,\ldots,h_N)$
* $W`_{N\times V}$ - матрица *word output* -  это тоже эмбеддинги но уже другие (они тоже обучаются в процессе)
* Выходной вектор $(y_1,\ldots,y_V)$ - скор для каждого слова из словаря размерности $V$

Мы видим два матричных перемножения - на самом деле W2V представляет собой очень простую нейронную сеть прямого распространения, *feed forward*.

На схеме видны две матрицы-скрытые слои. На самом деле это эмбеддинги контента, которые мы обучаем, каждая строка - эмбеддинг размерности N. Матрица эмбеддингов размером (ЧИСЛО СЛОВ В СЛОВАРЕ) X (РАЗМЕРНОСТЬ ЭМБЕДДИНГА) в начале обучения инициализируется рандомными числами, которые “превращаются” в осмысленные эмбеддинги, пока сеть обучается методом обратного распространения ошибки

На последнем слое мы получаем скоры для каждого слова из словаря. Скор (от англ score) с индексом i - это “уверенность” сети в том, что слово i может быть в контексте слова, которое мы прокидываем через сеть. То есть мы “кормим” сеть контекстом и уменьшаем лосс в случае, когда по контексту правильно удалось распознать слово внутри контекста. Слово с максимальным скором - это предсказание нашей сети. Зная "истинное" слово, которое мы предсказываем и то, что предсказала сеть, мы будем "подкручивать" веса эмбеддингов таким образом, чтобы лосс уменьшался и начинаем все лучше предсказывать слово по контексту.

Ниже показано, как работает, модификация *CBOW* - через нашу "сеть" пропускается каждое слово из контекста, мы пытаемся спрогнозировать слово "внутри" контекста:

![cbow](img/cbow.png)




В питоне существует модуль `gensim` который включает в себя библиотеки для обучения W2V.

Давайте применим алгоритм CBOW к нашему тексту:

In [22]:
from gensim.models import Word2Vec
import logging

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

texts = df.tokenized.values

model = Word2Vec(texts, vector_size=10, window=7, min_count=2, workers=4, epochs=10, sg=0)

2025-12-26 09:13:44,064 : INFO : collecting all words and their counts
2025-12-26 09:13:44,065 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2025-12-26 09:13:44,071 : INFO : collected 10795 word types from a corpus of 67165 raw words and 3904 sentences
2025-12-26 09:13:44,071 : INFO : Creating a fresh vocabulary
2025-12-26 09:13:44,076 : INFO : Word2Vec lifecycle event {'msg': 'effective_min_count=2 retains 3843 unique words (35.60% of original 10795, drops 6952)', 'datetime': '2025-12-26T09:13:44.076005', 'gensim': '4.4.0', 'python': '3.12.11 (main, Jun  3 2025, 15:41:47) [Clang 17.0.0 (clang-1700.0.13.3)]', 'platform': 'macOS-15.0-arm64-arm-64bit', 'event': 'prepare_vocab'}
2025-12-26 09:13:44,076 : INFO : Word2Vec lifecycle event {'msg': 'effective_min_count=2 leaves 60213 word corpus (89.65% of original 67165, drops 6952)', 'datetime': '2025-12-26T09:13:44.076275', 'gensim': '4.4.0', 'python': '3.12.11 (main, Jun  3 2025, 15:41:47) [Clang 17.0.0 (clang-

Мы обучили эмбеддинги слов. Давайте проверим, какой вектор обучился для слова `android`

Мы видим набор цифр - это вектор длины 10. Давайте найдём, какие слова соответствуют максимально похожим векторам


In [23]:
print(model.wv.get_vector('android'))
model.wv.most_similar('android')

[ 0.24677023 -0.41371143  1.2445815   0.56448156  1.7247869   1.844483
  2.8947337   1.9079707  -2.724034   -1.2670892 ]


[('version,', 0.9756479263305664),
 ('iphone,', 0.9746665358543396),
 ('share/gather', 0.9744194149971008),
 ('spots', 0.9653379917144775),
 ('chrome', 0.9650068283081055),
 ('elusive', 0.9646620750427246),
 ('blackberry', 0.963772177696228),
 ('our', 0.9613988995552063),
 ('&amp;', 0.9607152342796326),
 ('check', 0.9594240784645081)]


Мы видим, что модель обучила похожие вектора для слов `blackberry`, `iphone`  - это всё названия телефонов, то есть модель работает!

На основе векторизованных слов можно строить векторное описание целого предложения - такой алгоритм называется `doc2vec`.

Вывод: [модель W2V](http://www.1-4-5.net/~dmm/ml/how_does_word2vec_work.pdf) которая позволяет превращать схожие слова в "близкие" векторы, ориентируясь на контекст

# Прикладная задача - NER



**NOTE**: зарегистрируйтесь на HuggingFace, это классная платформа для публикации моделей, датасетов: [More models](https://huggingface.co/models?library=transformers&sort=trending&search=bert) - тут можно найти открытые модель

Важно: используется кеширование

In [25]:
from transformers import logging

from transformers import AutoModelForTokenClassification, AutoTokenizer, pipeline


logging.set_verbosity_error()

print('Model loading started...')
tokenizer = AutoTokenizer.from_pretrained(
    "dslim/distilbert-NER",
    cache_dir=os.path.join(root_data_dir, "models")
)
model = AutoModelForTokenClassification.from_pretrained(
    "dslim/distilbert-NER",
    cache_dir=os.path.join(root_data_dir, "models")
)

ner = pipeline(
    'ner', model=model, tokenizer=tokenizer,
    aggregation_strategy="simple"
)
print('Model loading finished!')

print(sample_text)
print()
ner(sample_text)[:5]

Model loading started...
Model loading finished!
SoftBank Vision Fund 2 is leading the round, a Series C, with iPod “father” and Nest co-founder Tony Fadell (by way of Future Shape), Blisce, French entrepreneur Xavier Niel, Mirabaud, Cassius and Evolution — all previous backers — also participating. (Previous investors in the company also include DeepMind co-founders Mustafa Suleyman and Demis Hassabis, notable given the company’s early focus on data science and recommendation algorithms.) Prior to this round Dice had raised around $45 million, according to PitchBook estimates.



[{'entity_group': 'ORG',
  'score': np.float32(0.98425573),
  'word': 'Soft',
  'start': 0,
  'end': 4},
 {'entity_group': 'ORG',
  'score': np.float32(0.9850637),
  'word': '##B',
  'start': 4,
  'end': 5},
 {'entity_group': 'ORG',
  'score': np.float32(0.9770343),
  'word': '##an',
  'start': 5,
  'end': 7},
 {'entity_group': 'ORG',
  'score': np.float32(0.8805127),
  'word': '##k Vision Fund 2',
  'start': 7,
  'end': 22},
 {'entity_group': 'MISC',
  'score': np.float32(0.61475396),
  'word': 'Series C',
  'start': 47,
  'end': 55}]

## Метрики оценки

### BLEU vs ROUGE: в чём разница?

**BLEU** (для translation) фокусируется на **precision**:
> "Сколько из сгенерированного правильно?"

**ROUGE** (для summarization) фокусируется на **recall**:
> "Сколько из важного было найдено?"

#### Пример
```
Reference: "Apple announced record earnings of $90B"

Candidate A: "Apple announced record earnings"
BLEU: High ✅ (всё правильно, но короткое → brevity penalty)
ROUGE: Low ❌ (пропущена важная инфа "$90B")

Candidate B: "Apple Inc. announced very strong record earnings of $90B"
BLEU: Low ❌ (много лишних слов)
ROUGE: High ✅ (вся важная информация есть)
```

**Формулы:**
```
BLEU = BP × exp(Σ wₙ log pₙ)
где pₙ = n-gram precision

ROUGE-N Recall = matched n-grams / n-grams в reference
```

**Типичные значения:**

| Метрика | Summarization | Translation |
|---------|---------------|-------------|
| BLEU | 0.2-0.4 | 0.4-0.6 |
| ROUGE-1 | 0.3-0.5 | — |
| ROUGE-2 | 0.1-0.25 | — |

📚 **Источники:**
- [BLEU: a Method for Automatic Evaluation of Machine Translation (Papineni et al., 2002)](https://aclanthology.org/P02-1040.pdf)
- [ROUGE: A Package for Automatic Evaluation of Summaries (Lin, 2004)](https://aclanthology.org/W04-1013/)
- [BERTScore: Evaluating Text Generation with BERT (Zhang et al., 2020)](https://arxiv.org/abs/1904.09675)

---

## 6. Perplexity: что это и зачем?

### Интуиция

**Perplexity** = насколько модель "удивлена" правильным ответом
```
Предложение: "The cat sat on the ___"

Хорошая модель:
  P(mat) = 0.80 → Perplexity ≈ 1.25 ✅ (почти уверена)

Плохая модель:
  P(mat) = 0.05 → Perplexity = 20 ❌ (сильно удивлена)
```

### Математика
```
Perplexity = 2^(Cross-Entropy)
           = 2^(-1/N × Σ log₂ P(wᵢ | context))
```

**Интерпретация:**

Perplexity = K означает:
> "Модель в среднем так же не уверена, как если бы выбирала из K равновероятных вариантов"

**Типичные значения:**
```
Perplexity = 1       → Идеальная (overfitting!)
Perplexity = 10-30   → Отличная модель (GPT-3)
Perplexity = 50-100  → Хорошая модель
Perplexity = 200+    → Плохая модель
```

📚 **Источники:**
- [A Mathematical Theory of Communication (Shannon, 1948)](https://people.math.harvard.edu/~ctm/home/text/others/shannon/entropy/entropy.pdf) — основы энтропии
- [Speech and Language Processing (Jurafsky & Martin)](https://web.stanford.edu/~jurafsky/slp3/) — глава 3 про N-gram LM и perplexity


## Next level: transformers for embeddings evaluation

`Transformer` - архитектура которая позволяет генерировать эмбеддинги с помощью нейросетей

Архитектура стала следующим большим шагом после LSTM используется повсеместно для обработки последовательностей

Рекомендую для просмотра: [Transfromers united](https://www.youtube.com/playlist?list=PLoROMvodv4rNiJRchCzutFw5ItR_Z27CM) от Stanford.

```shell
hf auth login
```

For local env run in console

```shell
SCRIPT=run_ner_pipeline.py make run-script
```

Помните: трансформеры это не серебрянная пуля, правило 'мусор на входе - мусор на выходе' остаётся актуальным. То есть препроцуссинг никто не отменял

## Next next level: ollama embeddings

```shell
curl http://localhost:11434/api/embeddings \
  -d '{
    "model": "granite4:350m",
    "prompt": "Your text to embed goes here"
  }' \
  -H "Content-Type: application/json"
```

python

```python
import requests
import json

# Endpoint and model configuration
url = "http://localhost:11434/api/embeddings"
data = {
    "model": "granite4:350m",
    "prompt": "Your text to embed goes here"
}

response = requests.post(url, json=data)
embedding = response.json().get("embedding")
```

## Google AI studio

[docs](https://ai.google.dev/gemini-api/docs/embeddings)

this is not works in Russia >>>> __try to get Gemini keys at [google AI studio](https://aistudio.google.com/)__ 